**This kernel is for applicated Machine Learning to predicting trip duration in NYC taxi**
**Please to check my kernel [NYC EDA](https://www.kaggle.com/baoanh/nyc-taxi-duration-eda-by-nguyen-khac-bao-anh) to understanding this dataset and knowing kind of dataset that i used for this kernel**

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# visualiser les données
import matplotlib.pyplot as plt
import seaborn as sns
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))
# Any results you write to the current directory are saved as output.

import warnings
warnings.filterwarnings('ignore')

['nyc-taxi-duration-eda-by-nguyen-khac-bao-anh', 'nyc-taxi-trip-duration']


In [2]:
%matplotlib inline
sns.set({'figure.figsize':(16,8)})

# Data Loading

In [3]:
train = pd.read_csv("../input/nyc-taxi-duration-eda-by-nguyen-khac-bao-anh/training_data.csv")
test = pd.read_csv("../input/nyc-taxi-duration-eda-by-nguyen-khac-bao-anh/testing_data.csv")

In [4]:
print(f"shape of training set{train.shape}")
print(f"shape of testing set{test.shape}")

shape of training set(1458644, 26)
shape of testing set(625134, 23)


### Data Training vs Data testing:

In [5]:
train.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,log_trip_duration,month,day,dayofweek,weekday,hour,minute,dist_long,dist_lat,dist,0_x,0_y,ave_speed_x,ave_speed_y,Temp.
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,0,455,6.120297,3,14,0,0,17,24,-0.017525,0.002335,0.017680,68221,72764,3.610834,3.898475,4.4
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,0,663,6.496775,6,12,6,6,0,43,0.019066,0.007412,0.020456,59478,51890,3.855183,3.548049,28.9
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,0,2124,7.661056,1,19,1,1,11,35,0.026306,0.053852,0.059934,111200,38071,3.474121,4.320430,-6.7
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,0,429,6.061457,4,6,2,2,19,32,0.002228,0.013252,0.013438,29991,38071,4.085823,4.320430,7.2
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,0,435,6.075346,3,26,5,5,13,30,-0.000130,0.010689,0.010690,36415,11401,4.407767,4.054702,9.4


In [6]:
test.head()

,id,vendor_id,pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,month,day,dayofweek,weekday,hour,minute,dist_long,dist_lat,dist,0_x,0_y,ave_speed_x,ave_speed_y,Temp.
0,id3004672,1,2016-06-30 23:59:58,1,-73.988129,40.732029,-73.990173,40.756680,0,6,30,3,3,23,59,0.002045,-0.024651,0.024735,73241,75286,3.698298,3.342108,24.4
1,id3505355,1,2016-06-30 23:59:53,1,-73.964203,40.679993,-73.959808,40.655403,0,6,30,3,3,23,59,-0.004395,0.024590,0.024979,473,1018,4.581356,5.379099,24.4
2,id1217141,1,2016-06-30 23:59:47,1,-73.997437,40.737583,-73.986160,40.729523,0,6,30,3,3,23,59,-0.011276,0.008060,0.013861,62747,58382,3.764735,3.558096,24.4
3,id2150126,2,2016-06-30 23:59:41,1,-73.956070,40.771900,-73.986427,40.730469,0,6,30,3,3,23,59,0.030357,0.041431,0.051363,73643,58382,3.901261,3.558096,24.4
4,id1598245,1,2016-06-30 23:59:33,1,-73.970215,40.761475,-73.961510,40.755890,0,6,30,3,3,23,59,-0.008705,0.005585,0.010343,128664,36023,3.844566,3.799226,24.4


# Select dataset

In [7]:
col_diff = list(set(train.columns).difference(set(test.columns)))
print(f"La différence de la variable entre data training et data testing:\
{set(train.columns).difference(set(test.columns))}")

La différence de la variable entre data training et data testing:{'dropoff_datetime', 'trip_duration', 'log_trip_duration'}


In [8]:
xtrain = train.drop(['id', 'pickup_datetime', 'dropoff_datetime', 'trip_duration', 'log_trip_duration'], axis = 1).as_matrix()
xtest = test.drop(['id', 'pickup_datetime', ], axis = 1).as_matrix()
y = train['log_trip_duration'].values
del(train, test)

In [9]:
from sklearn.model_selection import train_test_split, cross_val_score

In [10]:
X_train, X_valid, y_train, y_valid = train_test_split(xtrain,y, test_size=0.2, random_state=42)
X_train.shape, X_valid.shape, y_train.shape, y_valid.shape

((1166915, 21), (291729, 21), (1166915,), (291729,))

# Select model

In [11]:
#from sklearn.ensemble import RandomForestRegressor

In [12]:
#rf_defaut = RandomForestRegressor()
# crosse validation pour tester si le model est stable 
#rf_cv = cross_val_score(rf_defaut, X_train, y_train, cv=5)
#rf_cv

In [13]:
#plt.plot(range(1,len(rf_cv)+1), rf_cv)
#plt.ylim(np.min(rf_cv)-0.1,np.max(rf_cv)+0.1)
#plt.xlabel("nombre de fold")
#plt.ylabel("score du model Random Forest Regressor");

> le line chart au-dessus, il nous dit que ce model est stable, c'est à dire, ce model est capable de généraliser

In [14]:
# la fonction permet de nous donner un score qui est le règle de cette compétition
# (root mean squared log error)
#from sklearn.metrics import mean_squared_log_error, mean_squared_error
def rmse(y,pred):
    return np.sqrt(np.mean(np.square(np.log(np.exp(y))-np.log(np.exp(pred)))))

In [15]:
#rf_defaut = RandomForestRegressor()
#rf_defaut.fit(X_train, y_train)

In [16]:
#y_pred = rf_defaut.predict(X_valid)

In [17]:
#print(rmse(y_valid,y_pred))

## Grid search pour optimiser l'algorithme Random Forest

In [18]:
#from sklearn.model_selection import GridSearchCV

In [19]:
# n_estimators et max_depth pour fitter bien le model mais causer overfitting
# par contre, min_samples_leaf et min_samples_split nous permets de éviter overfitting en donnant la valeur
# plus grand
#params = {
#    'n_estimators' : [10, 15, 20],
#    'max_depth': [30, 50, 100],
#    'min_samples_leaf': [100],
#    'min_samples_split': [150]
#}
#rf2 = RandomForestRegressor()
#gs_rf2 = GridSearchCV(rf2, param_grid=params, scoring='neg_mean_squared_error',cv=3, verbose=10, n_jobs=-1)
#gs_rf2.fit(X_train, y_train)
#gs_rf2.best_score_
#best_rf2 = gs_rf2.best_estimator_

In [20]:
#rf2 = RandomForestRegressor(n_estimators=10,min_samples_leaf=100, min_samples_split=150)

In [21]:
#rf2_cv = cross_val_score(rf2, X_train, y_train, cv=5)
#rf2_cv

In [22]:
#plt.plot(range(1,len(rf2_cv)+1), rf2_cv)
#plt.ylim(np.min(rf2_cv)-0.1,np.max(rf2_cv)+0.1)
#plt.xlabel("nombre de fold")
#plt.ylabel("score du model Random Forest Regressor avec hyperparameters");

In [23]:
#rf2.fit(X_train, y_train)

In [24]:
#pred = rf2.predict(X_valid)

In [25]:
#print(rmse(y_valid,pred))

# LightGBM pour prédire trip duration

In [26]:
import lightgbm as lgb

In [27]:
#lgb_train = lgb.Dataset(X_train, y_train)
#lgb_valid = lgb.Dataset(X_valid, y_valid)
# training all dataset
dtrain = lgb.Dataset(xtrain,y)
del(X_train, y_train, X_valid, y_valid, xtrain,y)

In [28]:
lgb_params = {
    'learning_rate': 0.1,
    'max_depth': 8,
    'num_leaves': 55, 
    'objective': 'regression',
    #'metric': {'rmse'},
    'feature_fraction': 0.9,
    'bagging_fraction': 0.5,
    #'bagging_freq': 5,
    'max_bin': 300}       # 1000

In [29]:
#cv_result_lgb = lgb.cv(lgb_params,
#                       lgb_train, 
#                       num_boost_round=1000, 
#                       nfold=3,
#                       early_stopping_rounds=50, 
#                       verbose_eval=100, 
#                       show_stdv=True,stratified=False)

In [30]:
#n_rounds = len(cv_result_lgb['rmsle-mean'])
#print('num_boost_rounds_lgb=' + str(n_rounds))

In [31]:
# visualisation des résultat dans cv
# CV scores
#train_scores = np.array(cv_result_lgb['rmsle-mean'])
#train_stds = np.array(cv_result_lgb['rmsle-stdv'])
#plt.plot(train_scores, color='violet')
#plt.fill_between(range(len(cv_result_lgb['rmsle-mean'])), 
#                 train_scores - train_stds, train_scores + train_stds, 
#                 alpha=0.1, color='violet')
#plt.title('LightGMB CV-results')
#plt.xlabel("number of rounds")
#plt.ylabel("score");

In [32]:
# Train a model
#model_lgb = lgb.train(lgb_params, 
#                      dtrain, 
#                      feval=lgb_rmsle_score, 
#                      num_boost_round=n_rounds)

In [33]:
## Predict on train
#y_train_pred = model_lgb.predict(X_train)
#print('RMSLE on train = {}'.format(rmse(y_train_pred, y_train)))
## Predict on validation
#y_valid_pred = model_lgb.predict(X_valid)
#print('RMSLE on valid = {}'.format(rmse(y_valid_pred, y_valid)))

In [34]:
# Train a model
model_lgb = lgb.train(lgb_params, 
                      dtrain,
                      num_boost_round=1500)

# submission on kaggle

In [35]:
submit = pd.read_csv('../input/nyc-taxi-trip-duration/sample_submission.csv')
submit.head()

,id,trip_duration
0,id3004672,959
1,id3505355,959
2,id1217141,959
3,id2150126,959
4,id1598245,959


In [36]:
pred_test = np.exp(model_lgb.predict(xtest))

In [37]:
submit['trip_duration'] = pred_test

In [38]:
submit.head()

,id,trip_duration
0,id3004672,799.311700
1,id3505355,651.977997
2,id1217141,417.724045
3,id2150126,985.922494
4,id1598245,354.242444


In [39]:
submit.to_csv("submit_file.csv", index=False)